In [1]:
import os

os.environ.update({"TF_CPP_MIN_LOG_LEVEL": "3", "CUDA_VISIBLE_DEVICES": ""})

import time

import tensorflow as tf

import chiron


In [2]:
dataset = chiron.tfrecord.load_tfrecord("../tfrecord/combined/train.tfrecord")
dataset.element_spec


(TensorSpec(shape=(None, None, None), dtype=tf.float32, name=None),
 TensorSpec(shape=(), dtype=tf.string, name=None))

In [3]:
label_map = {"meningioma": 0, "glioma": 0, "pituitary": 0, "notumor": 1}
image_size = [224, 224]
batch_size = 32

label_mapper = chiron.preprocessing.LabelMapper(label_map)
resizer = chiron.preprocessing.Resizer(image_size)
standardizer = chiron.preprocessing.PerImageStandardizer()
grayscale_to_rgb = chiron.preprocessing.GrayscaleToRgb()


In [4]:
def benchmark(dataset):
    start_time = time.perf_counter()
    for _ in range(2):
        for _ in dataset:
            time.sleep(0.01)
    print(time.perf_counter() - start_time)


benchmark(
    dataset.map(label_mapper)
    .map(resizer)
    .batch(batch_size)
    .map(standardizer)
    .map(grayscale_to_rgb)
)


10.383399900048971


In [5]:
benchmark(
    dataset.map(label_mapper, num_parallel_calls=tf.data.AUTOTUNE)
    .map(resizer, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size)
    .map(standardizer, num_parallel_calls=tf.data.AUTOTUNE)
    .map(grayscale_to_rgb, num_parallel_calls=tf.data.AUTOTUNE)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)


3.443941699981224
